In [1]:
import os
import requests
from langchain.document_loaders import TextLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer

# Step 1: Define the function to load documents
def load_docs(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):  # Only load .txt files
            filepath = os.path.join(directory, filename)
            try:
                # Use TextLoader to load each file
                loader = TextLoader(filepath)
                documents.extend(loader.load())  # Add the loaded documents to the list
                print(f"Loaded {filename}")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
    return documents

directory = 'D:\\open_llama'
documents = load_docs(directory)

# Check how many documents were loaded
print(f"Loaded {len(documents)} documents")
print(documents)

c:\Users\Balas\anaconda3\envs\ragtr\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Loaded sample.txt
Loaded 1 documents
[Document(metadata={'source': 'D:\\open_llama\\sample.txt'}, page_content='Day 1 01-07-2024 08:00 | Crowd | "The beach is moderately crowded this morning, families are enjoying the cool breeze."\n01-07-2024 10:45 | Road | "Roads leading to Sunrise Point are partially blocked due to heavy fog. Drive carefully."\n01-07-2024 12:15 | Event | "A small food festival has started near the town hall with local delicacies."\n01-07-2024 13:00 | Weather | "Clear skies with a gentle breeze, perfect for outdoor activities."\n01-07-2024 15:30 | Road | "Main roads around the coastal area are congested with traffic due to construction work."\n01-07-2024 17:00 | Event | "Live music performance is scheduled at the central square at 6 PM today."\n01-07-2024 21:00 | Road | "Night-time repairs are ongoing at Riverside Road. Delays are expected."\n01-07-2024 22:00 | Weather | "Temperature drops expected, carry a light jacket."\nDay 2 02-07-2024 07:45 | Crowd | "Few people

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# Define the split_docs function to split text into smaller chunks
def split_docs(documents, chunk_size=100, chunk_overlap=20):
    # Create a text splitter instance
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    # Extract the page_content from documents and split it
    all_docs = []
    for doc in documents:
        # Split the page_content of each document into chunks
        chunks = text_splitter.split_text(doc.page_content)  # Use doc.page_content to access the text content
        all_docs.extend(chunks)  # Add the chunks to the all_docs list
    return all_docs


processed_docs = split_docs(documents)

# Output the processed documents (chunks of text)
print(f"Processed {len(processed_docs)} chunks.")
print(processed_docs)  

Processed 146 chunks.
['Day 1 01-07-2024 08:00 | Crowd | "The beach is moderately crowded this morning, families are', 'families are enjoying the cool breeze."', '01-07-2024 10:45 | Road | "Roads leading to Sunrise Point are partially blocked due to heavy fog.', 'due to heavy fog. Drive carefully."', '01-07-2024 12:15 | Event | "A small food festival has started near the town hall with local', 'hall with local delicacies."', '01-07-2024 13:00 | Weather | "Clear skies with a gentle breeze, perfect for outdoor activities."', '01-07-2024 15:30 | Road | "Main roads around the coastal area are congested with traffic due to', 'with traffic due to construction work."', '01-07-2024 17:00 | Event | "Live music performance is scheduled at the central square at 6 PM', 'square at 6 PM today."', '01-07-2024 21:00 | Road | "Night-time repairs are ongoing at Riverside Road. Delays are expected."', '01-07-2024 22:00 | Weather | "Temperature drops expected, carry a light jacket."', 'Day 2 02-07-2024 07

In [3]:
print(processed_docs)

['Day 1 01-07-2024 08:00 | Crowd | "The beach is moderately crowded this morning, families are', 'families are enjoying the cool breeze."', '01-07-2024 10:45 | Road | "Roads leading to Sunrise Point are partially blocked due to heavy fog.', 'due to heavy fog. Drive carefully."', '01-07-2024 12:15 | Event | "A small food festival has started near the town hall with local', 'hall with local delicacies."', '01-07-2024 13:00 | Weather | "Clear skies with a gentle breeze, perfect for outdoor activities."', '01-07-2024 15:30 | Road | "Main roads around the coastal area are congested with traffic due to', 'with traffic due to construction work."', '01-07-2024 17:00 | Event | "Live music performance is scheduled at the central square at 6 PM', 'square at 6 PM today."', '01-07-2024 21:00 | Road | "Night-time repairs are ongoing at Riverside Road. Delays are expected."', '01-07-2024 22:00 | Weather | "Temperature drops expected, carry a light jacket."', 'Day 2 02-07-2024 07:45 | Crowd | "Few peo

In [4]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [5]:
# Initialize Pinecone
pc = Pinecone(api_key="#############")
index = pc.Index("travel")

# Initialize SentenceTransformer for generating embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
def generate_and_upsert(chunk):
    try:
        # Generate embedding for the chunk
        embedding = embedding_model.encode([chunk])[0]
        
        # Generate a unique ID for the chunk
        unique_id = str(np.random.randint(1e6))  # Or use UUID for more uniqueness
        
        # Prepare the data for upsert (embedding + metadata if needed)
        upsert_data = [(unique_id, embedding)]
        
        # Upsert the embedding to Pinecone
        index.upsert(vectors=upsert_data)
        print(f"Successfully upserted chunk with ID: {unique_id}")
    except Exception as e:
        print(f"Error processing chunk: {chunk}. Error: {e}")

In [7]:
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(generate_and_upsert, processed_docs)

c:\Users\Balas\anaconda3\envs\ragtr\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Successfully upserted chunk with ID: 637100
Successfully upserted chunk with ID: 913370
Successfully upserted chunk with ID: 276757
Successfully upserted chunk with ID: 754648
Successfully upserted chunk with ID: 721552
Successfully upserted chunk with ID: 634832
Successfully upserted chunk with ID: 562409
Successfully upserted chunk with ID: 259782
Successfully upserted chunk with ID: 307972
Successfully upserted chunk with ID: 556430
Successfully upserted chunk with ID: 55138
Successfully upserted chunk with ID: 454327
Successfully upserted chunk with ID: 672979
Successfully upserted chunk with ID: 784723
Successfully upserted chunk with ID: 761179
Successfully upserted chunk with ID: 758333
Successfully upserted chunk with ID: 49337
Successfully upserted chunk with ID: 736836
Successfully upserted chunk with ID: 30883
Successfully upserted chunk with ID: 379387
Successfully upserted chunk with ID: 304821
Successfully upserted chunk with ID: 633845
Successfully upserted chunk with ID

In [8]:
# Define the query text
query_text = "What is the weather forecast for today?"  # Example query

# Generate embedding for the query (ensure it's a list of floats)
query_embedding = embedding_model.encode([query_text])[0]  # Generate embedding for the query
query_embedding = query_embedding.tolist()  # Convert numpy array to list

# Query Pinecone for similar vectors (ensure to specify the correct namespace)
query_response = index.query(
    top_k=5,  # Number of similar chunks to retrieve
    include_metadata=True,  # Include metadata in the results
    vector=query_embedding  # Query vector to find the most similar vectors
)

# Step 2: Process the query response and print the results
for match in query_response['matches']:
    vector_id = match['id']
    print(f"ID: {vector_id}")  # ID of the matching chunk
    print(f"Score: {match['score']}")  # Similarity score (relevance of the match)

    # Fetch the vector (embedding) using the vector_id
    query_r = index.fetch([vector_id])

    # Check if the vector is present in the fetched response
    if 'vectors' in query_r and vector_id in query_r['vectors']:
        fetched_embedding = query_r['vectors'][vector_id]
        print(f"Fetched embedding for vector ID {vector_id}:", fetched_embedding)
    else:
        print(f"Vector ID {vector_id} not found.")

    # # Check if metadata is available
    # if 'metadata' in match:
    #     print(f"Metadata: {match['metadata']}")  # Metadata associated with the chunk
    # else:
    #     print("Metadata: No metadata available")

    print()


ID: 712182
Score: 0.646508276
Fetched embedding for vector ID 712182: {'id': '712182',
 'values': [-0.05630688,
            0.0656032786,
            0.0830761865,
            0.107364491,
            0.0540323183,
            -0.0351539515,
            -0.0563064776,
            -0.0326194167,
            0.00492599653,
            -0.0184354018,
            -0.0213662256,
            -0.00648139557,
            -0.0473737046,
            -0.0399673954,
            -0.00846152753,
            0.0660760552,
            -0.00708302669,
            -0.140889838,
            -0.0618327148,
            0.034289144,
            -0.0649750903,
            0.0355883352,
            -0.0404249877,
            0.0495666675,
            0.0468351468,
            0.0118721658,
            -0.0544195957,
            -0.0331115164,
            -0.0190304592,
            0.0972864553,
            -0.0513365828,
            0.0536609255,
            -0.0308375172,
            -0.0172912497,
         

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pinecone

# Assuming index is initialized and connected to Pinecone
# Assuming embedding_model is initialized (e.g., from sentence-transformers)

def generate_response(query_text):
    # Step 1: Generate embedding for the query
    query_embedding = embedding_model.encode([query_text])[0]  # Generate embedding for the query
    query_embedding = query_embedding.tolist()  # Convert numpy array to list
    
    # Query Pinecone for similar vectors
    query_response = index.query(
        top_k=5,  
        vector=query_embedding,  
        include_metadata=False  
    )
    
    # Step 2: Prepare the context
    context = ""
    for match in query_response['matches'][:3]:  
        vector_id = match['id']
        query_r = index.fetch([vector_id])
        if 'vectors' in query_r and vector_id in query_r['vectors']:
            fetched_embedding = query_r['vectors'][vector_id]['values']
            context += f"Vector ID: {vector_id} Embedding (truncated): {fetched_embedding[:10]}... \n\n"
    
    # Step 3: Format the prompt
    prompt = (
        f"Context:\n{context}\n\n"
        f"Question: {query_text}\n"
        f"Answer:"
    )

    # Step 4: Generate a response using GPT-2
    model_name = "gpt2"
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token  # Set pad_token_id to eos_token_id
    
    # Tokenize the input prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = inputs != tokenizer.pad_token_id  # Attention mask for reliable results

    # Generate a response
    output = model.generate(
        inputs, 
        attention_mask=attention_mask, 
        max_length=500, 
        num_return_sequences=1, 
        no_repeat_ngram_size=2, 
        top_k=50, 
        top_p=0.95, 
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # Avoid warnings
    )

    # Decode and extract the answer part
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    answer_start = generated_response.find("Answer:") + len("Answer:")
    answer = generated_response[answer_start:].strip().split("\n")[0]  # Extract first line after "Answer:"
    
    return answer

# Example usage
user_query = "How is weather today?"
response = generate_response(user_query)
#print("Generated Response:", response)


c:\Users\Balas\anaconda3\envs\ragtr\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Balas\anaconda3\envs\ragtr\lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [10]:
import gradio as gr

# Dummy function for RAG chatbot - Replace with your actual RAG function
# def rag_chatbot(prompt):
#     """
#     Simulates a RAG chatbot response.
#     Replace this with your actual RAG chatbot implementation.
#     """
#     response = f"RAG Response: You asked '{prompt}'"
#     return response
# user_query = input()
# response = generate_response(user_query)
# Gradio Interface
demo = gr.Interface(
    fn=generate_response,                # Function to call
    inputs=gr.Textbox(             # Input text box for user queries
        placeholder="Ask me anything...",
        label="Chat with RAG Chatbot"
    ),
    outputs=gr.Textbox(label="Response"),  # Output response
    title="RAG Chatbot Demo",      # Interface title
    description="This is a simple demo of a Retrieval-Augmented Generation (RAG) chatbot."
)

# Launch the Interface
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [11]:
# # Example: Query for a specific vector by its unique ID
# vector_id = "400773"  # Replace with an actual ID of a vector you inserted

# # Query the vector by ID
# query_response = index.fetch([vector_id])

# # Check the response
# if query_response['vectors']:
#     fetched_embedding = query_response['vectors'][vector_id]
#     print(f"Fetched embedding for vector ID {vector_id}:", fetched_embedding)
# else:
#     print(f"Vector ID {vector_id} not found.")


c:\Users\Balas\anaconda3\envs\ragtr\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Balas\anaconda3\envs\ragtr\lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
